# Exploración del Dataset: DISCAPACIDAD


Importación de Librerías

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings

# Configuración
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print("✅ Librerías importadas correctamente")

Carga del Dataset

In [ ]:
# Definir rutas de forma robusta
import os
from pathlib import Path

# Obtener la ubicación del notebook
notebook_dir = Path.cwd()
print(f"Directorio actual: {notebook_dir}")

# Construir ruta al directorio data/raw
if 'notebooks' in str(notebook_dir):
    DATA_RAW_PATH = Path('../../data/raw')
    DICT_PATH = Path('../../data/diccionarios')
else:
    DATA_RAW_PATH = Path(r'E:\MTPE\data\raw')
    DICT_PATH = Path(r'E:\MTPE\data\diccionarios')

print(f"Ruta DATA_RAW_PATH: {DATA_RAW_PATH.resolve()}")
print(f"Directorio existe: {DATA_RAW_PATH.exists()}")

# Cargar dataset
archivo_csv = DATA_RAW_PATH / 'Dataset_DISCAPACIDAD.csv'
print(f"Archivo CSV: {archivo_csv.name}")
print(f"Archivo existe: {archivo_csv.exists()}")

if archivo_csv.exists():
    df_discapacidad = pd.read_csv(archivo_csv, encoding='utf-8-sig')
    print(f"\nDataset cargado exitosamente")
    print(f"Dimensiones: {df_discapacidad.shape[0]:,} filas x {df_discapacidad.shape[1]} columnas")
else:
    print(f"\nERROR: No se encuentra el archivo en: {archivo_csv.resolve()}")
    print(f"   Por favor, verifica la ruta del archivo.")

Vista Previa - Primeras Filas

In [ ]:
print(" Primeras 20 filas del dataset:")
display(df_discapacidad.head(20))

Vista Previa - Últimas Filas

In [ ]:
print("Últimas 10 filas del dataset:")
display(df_discapacidad.tail(10))

Vista Previa - Muestra Aleatoria

In [ ]:
print("Muestra aleatoria de 15 registros:")
display(df_discapacidad.sample(15, random_state=42))

Información General del Dataset

In [ ]:
print("Información del Dataset:")
print("=" * 80)
df_discapacidad.info()

print("\nColumnas del Dataset:")
print("=" * 80)
for idx, col in enumerate(df_discapacidad.columns, 1):
    print(f"{idx:2d}. {col} ({df_discapacidad[col].dtype})")

Análisis de Valores Nulos

In [ ]:
print("Análisis de Valores Nulos:")
print("=" * 80)

null_data = pd.DataFrame({
    'Columna': df_discapacidad.columns,
    'Nulos': df_discapacidad.isnull().sum(),
    '% Nulos': (df_discapacidad.isnull().sum() / len(df_discapacidad) * 100).round(2)
}).sort_values(by='Nulos', ascending=False)

display(null_data)

if null_data['Nulos'].sum() > 0:
    plt.figure(figsize=(12, 6))
    null_cols = null_data[null_data['Nulos'] > 0]
    plt.barh(null_cols['Columna'], null_cols['% Nulos'])
    plt.xlabel('Porcentaje de Valores Nulos (%)')
    plt.title('Valores Nulos por Columna - DISCAPACIDAD')
    plt.tight_layout()
    plt.show()
else:
    print("No hay valores nulos en el dataset")

Análisis de Duplicados

In [ ]:
print("Análisis de Duplicados:")
print("=" * 80)

duplicados_totales = df_discapacidad.duplicated().sum()
print(f"Registros duplicados (completos): {duplicados_totales:,} ({duplicados_totales/len(df_discapacidad)*100:.2f}%)")

# Duplicados por DBIDPOSTULANTE
if 'DBIDPOSTULANTE' in df_discapacidad.columns:
    duplicados_id = df_discapacidad['DBIDPOSTULANTE'].duplicated().sum()
    print(f"Duplicados en DBIDPOSTULANTE: {duplicados_id:,} ({duplicados_id/len(df_discapacidad)*100:.2f}%)")

    if duplicados_id > 0:
        print("\nEjemplos de IDs duplicados (esto es esperado - un postulante puede tener múltiples discapacidades):")
        ids_duplicados = df_discapacidad[df_discapacidad['DBIDPOSTULANTE'].duplicated(keep=False)]['DBIDPOSTULANTE'].value_counts().head(10)
        display(ids_duplicados)

Estadísticas Descriptivas - Variables Numéricas

In [ ]:
print("Estadísticas Descriptivas - Variables Numéricas:")
print("=" * 80)
display(df_discapacidad.describe())

Estadísticas Descriptivas - Variables Categóricas

In [ ]:
print("Estadísticas Descriptivas - Variables Categóricas:")
print("=" * 80)
display(df_discapacidad.describe(include=['object']))

Análisis de Cardinalidad

In [ ]:
print("Análisis de Cardinalidad:")
print("=" * 80)

# Calcular valores únicos para cada columna
valores_unicos = [df_discapacidad[col].nunique() for col in df_discapacidad.columns]
tipos_dato = [str(df_discapacidad[col].dtype) for col in df_discapacidad.columns]
porcentaje_cardinalidad = [(df_discapacidad[col].nunique() / len(df_discapacidad) * 100) for col in df_discapacidad.columns]

cardinalidad = pd.DataFrame({
    'Columna': df_discapacidad.columns,
    'Valores_Únicos': valores_unicos,
    'Tipo_Dato': tipos_dato,
    '% Cardinalidad': [round(p, 2) for p in porcentaje_cardinalidad]
}).sort_values(by='Valores_Únicos', ascending=False)

display(cardinalidad)

Distribución de Variables Categóricas

In [ ]:
print("Distribución de Variables Categóricas:")
categorical_cols = df_discapacidad.select_dtypes(include=['object']).columns

for col in categorical_cols:
    print(f"\n{'='*80}")
    print(f"Columna: {col}")
    print(f"{'='*80}")
    value_counts = df_discapacidad[col].value_counts()
    print(f"Valores únicos: {df_discapacidad[col].nunique()}")
    print(f"\nDistribución completa:")
    display(pd.DataFrame({
        'Valor': value_counts.index,
        'Frecuencia': value_counts.values,
        'Porcentaje': (value_counts.values / len(df_discapacidad) * 100).round(2)
    }))

Análisis de CAUSA

In [ ]:
if 'CAUSA' in df_discapacidad.columns:
    print("Análisis de la variable CAUSA:")
    print("=" * 80)
    causa_counts = df_discapacidad['CAUSA'].value_counts()

    print(f"\nTotal de causas únicas: {df_discapacidad['CAUSA'].nunique()}")

    display(pd.DataFrame({
        'Causa': causa_counts.index,
        'Cantidad': causa_counts.values,
        'Porcentaje': (causa_counts.values / len(df_discapacidad) * 100).round(2)
    }))

    # Visualización
    plt.figure(figsize=(12, 6))
    plt.barh(causa_counts.index, causa_counts.values)
    plt.xlabel('Frecuencia')
    plt.ylabel('Causa')
    plt.title('Distribución de Causas de Discapacidad')
    plt.tight_layout()
    plt.show()

    # Gráfico de pastel
    plt.figure(figsize=(10, 8))
    plt.pie(causa_counts.values, labels=causa_counts.index, autopct='%1.1f%%', startangle=90)
    plt.title('Distribución Porcentual de Causas de Discapacidad')
    plt.axis('equal')
    plt.tight_layout()
    plt.show()

Análisis de DSCORE (Score de Discapacidad)

In [ ]:
if 'DSCORE' in df_discapacidad.columns:
    print("Análisis de la variable DSCORE:")
    print("=" * 80)

    print(f"DSCORE mínimo: {df_discapacidad['DSCORE'].min()}")
    print(f"DSCORE máximo: {df_discapacidad['DSCORE'].max()}")
    print(f"DSCORE promedio: {df_discapacidad['DSCORE'].mean():.2f}")
    print(f"DSCORE mediana: {df_discapacidad['DSCORE'].median()}")
    print(f"Desviación estándar: {df_discapacidad['DSCORE'].std():.2f}")

    print(f"\nDistribución de valores de DSCORE:")
    dscore_counts = df_discapacidad['DSCORE'].value_counts().sort_index()
    display(pd.DataFrame({
        'DSCORE': dscore_counts.index,
        'Frecuencia': dscore_counts.values,
        'Porcentaje': (dscore_counts.values / len(df_discapacidad) * 100).round(2)
    }))

    # Visualización
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))

    # Histograma
    axes[0].hist(df_discapacidad['DSCORE'].dropna(), bins=20, edgecolor='black')
    axes[0].set_xlabel('DSCORE')
    axes[0].set_ylabel('Frecuencia')
    axes[0].set_title('Distribución de DSCORE')
    axes[0].axvline(df_discapacidad['DSCORE'].mean(), color='red', linestyle='--', label=f'Media: {df_discapacidad["DSCORE"].mean():.1f}')
    axes[0].legend()

    # Boxplot
    axes[1].boxplot(df_discapacidad['DSCORE'].dropna())
    axes[1].set_ylabel('DSCORE')
    axes[1].set_title('Boxplot de DSCORE')

    plt.tight_layout()
    plt.show()

Análisis de DBIDPOSTULANTE (IDs Únicos)

In [ ]:
if 'DBIDPOSTULANTE' in df_discapacidad.columns:
    print("Análisis de DBIDPOSTULANTE:")
    print("=" * 80)

    postulantes_unicos = df_discapacidad['DBIDPOSTULANTE'].nunique()
    total_registros = len(df_discapacidad)

    print(f"Total de postulantes únicos: {postulantes_unicos:,}")
    print(f"Total de registros: {total_registros:,}")
    print(f"Promedio de registros por postulante: {total_registros/postulantes_unicos:.2f}")

    # Distribución de discapacidades por postulante
    discapacidades_por_persona = df_discapacidad['DBIDPOSTULANTE'].value_counts()

    print(f"\n📊 Distribución de número de discapacidades por postulante:")
    dist_disc = discapacidades_por_persona.value_counts().sort_index()
    display(pd.DataFrame({
        'Num_Discapacidades': dist_disc.index,
        'Cantidad_Postulantes': dist_disc.values,
        'Porcentaje': (dist_disc.values / postulantes_unicos * 100).round(2)
    }))

    # Visualización
    plt.figure(figsize=(10, 6))
    plt.bar(dist_disc.index.astype(str), dist_disc.values)
    plt.xlabel('Número de Discapacidades')
    plt.ylabel('Cantidad de Postulantes')
    plt.title('Distribución de Número de Discapacidades por Postulante')
    plt.tight_layout()
    plt.show()

 Análisis Cruzado: CAUSA vs DSCORE

In [ ]:
if all(col in df_discapacidad.columns for col in ['CAUSA', 'DSCORE']):
    print("Análisis Cruzado: CAUSA vs DSCORE")
    print("=" * 80)

    # Tabla cruzada
    tabla_cruzada = pd.crosstab(df_discapacidad['CAUSA'], df_discapacidad['DSCORE'], margins=True)
    display(tabla_cruzada)

    # Promedio de DSCORE por CAUSA
    print("\n Promedio de DSCORE por CAUSA:")
    promedio_dscore = df_discapacidad.groupby('CAUSA')['DSCORE'].agg(['mean', 'median', 'std', 'count']).round(2)
    promedio_dscore.columns = ['Promedio', 'Mediana', 'Desv_Std', 'Cantidad']
    display(promedio_dscore)

    # Visualización
    plt.figure(figsize=(12, 6))
    df_discapacidad.boxplot(column='DSCORE', by='CAUSA', figsize=(12, 6))
    plt.xlabel('Causa')
    plt.ylabel('DSCORE')
    plt.title('Distribución de DSCORE por Causa de Discapacidad')
    plt.suptitle('')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()